In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [2]:
bs_path = "Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_937/Processing_v2/air_937_business_shell_v31_baseline_201506_results_w20230727-060927.csv"
# test1_path = "Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_937/Processing_v2/air_937_business_shell_v31_test1_201506_results_w20230727-085608.csv"
test1_path = 'Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_937/Processing_v3/air_937_business_shell_v31_test1_201506_results_w20230729-140223.csv'
test2_path = "Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_937/Processing_v2/air_937_business_shell_v31_test2_201506_results_w20230727-133356.csv"

In [3]:
string_list = {'lnlexidsele': str, 'outbestbusname': str, 'outbestbusstreetaddr': str, 'outbestbuscity': str, 'outbestbusstate': str, 'outbestbuszip': str, 
               'outbestbustin': str, 'outbestbusphone': str, 'outbestnaics': str, 'outbestsic': str, 'liennewesttype': str, 'judgmentnewesttype': str, 
               'inputaddrtype': str, }

In [4]:
baseline = pd.read_adls(bs_path, reader = pd.read_csv, encoding='iso-8859-1', dtype = string_list, verbose = False, low_memory = False)
print(baseline.shape)
test_1 = pd.read_adls(test1_path, reader = pd.read_csv, encoding='iso-8859-1', dtype = string_list, verbose = False, low_memory = False)
print(test_1.shape)
test_2 = pd.read_adls(test2_path, reader = pd.read_csv, encoding='iso-8859-1', dtype = string_list, verbose = False, low_memory = False)
print(test_2.shape)

(100000, 2304)
(99998, 2304)
(100000, 2304)


In [5]:
bs_seleid = baseline[["accountnumber", "seleid"]]
bs_seleid.columns = ["accountnumber", "bs_seleid"]
test_1 = test_1.merge(bs_seleid, on = "accountnumber")
print(test_1.shape)
test_1 = test_1[test_1["seleid"] == test_1["bs_seleid"]]
print(test_1.shape)

test_2 = test_2.merge(bs_seleid, on = "accountnumber")
print(test_2.shape)
test_2 = test_2[test_2["seleid"] == test_2["bs_seleid"]]
print(test_2.shape)

(99998, 2305)
(99998, 2305)
(100000, 2305)
(100000, 2305)


In [16]:
# keep = ['accountnumber', 'model1score', 'model2score']
# test_1 = test_1[keep]
# test_1.columns = ["t1_"+x for x in test_1.columns]
# test_2 = test_2[keep]
# test_2.columns = ["t2_"+x for x in test_2.columns]
# baseline = baseline[keep]
# baseline.columns = ["b_"+x for x in keep]
# merged = test_1.merge(test_2, left_on = "t1_accountnumber", right_on = "t2_accountnumber")
# merged = merged.merge(baseline, left_on = "t1_accountnumber", right_on = "b_accountnumber")
# print(merged.shape)
merged["t1_b_mld1score"] = np.where((merged.t1_model1score >500) & (merged.b_model1score >500), merged.t1_model1score - merged.b_model1score, -999_999)
merged["t1_b_mld2score"] = np.where((merged.t1_model2score >500) & (merged.b_model2score >500), merged.t1_model2score - merged.b_model2score, -999_999)
merged["t2_b_mld1score"] = np.where((merged.t2_model1score >500) & (merged.b_model1score >500), merged.t2_model1score - merged.b_model1score, -999_999)
merged["t2_b_mld2score"] = np.where((merged.t2_model2score >500) & (merged.b_model2score >500), merged.t2_model2score - merged.b_model2score, -999_999)

In [17]:
fmt = make_format(cuts = [-np.inf, -51, -41, -31, -21, -11, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-999_999])
result = dict()

result["f1"] = freq(merged["t1_b_mld1score"], format = fmt, observed = True)
result["f2"] = freq(merged["t1_b_mld2score"], format = fmt, observed = True)
result["f3"] = freq(merged["t2_b_mld1score"], format = fmt, observed = True)
result["f4"] = freq(merged["t2_b_mld2score"], format = fmt, observed = True)

wb = TableWriter(filename = "./_temp/freq_201506.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[1],
    )
wb.close()

In [6]:
keep = ["accountnumber", "sbfetimeoldestcycle", "sbfeaccountcount", "model2score", "sbfehitindex"]
temp_1 = test_1[keep].copy()
temp_1.columns = ["t1_" + x for x in temp_1.columns]
temp_2 = test_2[keep].copy()
merged = temp_1.merge(temp_2, left_on = "t1_accountnumber", right_on = "accountnumber")
merged["sbfetimeoldestcycle_flag"] = np.where(merged.sbfetimeoldestcycle != merged.t1_sbfetimeoldestcycle, 1, 0)
merged["sbfeaccountcount_flag"] = np.where(merged.sbfeaccountcount != merged.t1_sbfeaccountcount, 1, 0)
merged["model2score_flag"] = np.where(merged.model2score != merged.t1_model2score, 1, 0)
merged["sbfehitindex_flag"] = np.where(merged.sbfehitindex != merged.sbfehitindex, 1, 0)

In [7]:
merged.sbfetimeoldestcycle.describe([0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95])

count    99988.000000
mean       -67.153758
std         67.178698
min        -99.000000
5%         -99.000000
10%        -99.000000
20%        -99.000000
30%        -99.000000
40%        -99.000000
50%        -99.000000
60%        -99.000000
70%        -99.000000
80%        -99.000000
90%         88.000000
95%         95.000000
max         96.000000
Name: sbfetimeoldestcycle, dtype: float64

In [8]:
merged.t1_sbfetimeoldestcycle.describe([0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95])

count    99988.0
mean       -99.0
std          0.0
min        -99.0
5%         -99.0
10%        -99.0
20%        -99.0
30%        -99.0
40%        -99.0
50%        -99.0
60%        -99.0
70%        -99.0
80%        -99.0
90%        -99.0
95%        -99.0
max        -99.0
Name: t1_sbfetimeoldestcycle, dtype: float64

In [9]:
freq(merged["sbfetimeoldestcycle_flag"])

,Count,Pct,Cuml Count,Cuml Pct
sbfetimeoldestcycle_flag,,,,
0,80921.0,0.809226,80921.0,0.809226
1,19077.0,0.190774,99998.0,1.000000


In [10]:
freq(merged["sbfeaccountcount_flag"])

,Count,Pct,Cuml Count,Cuml Pct
sbfeaccountcount_flag,,,,
0,80921.0,0.809226,80921.0,0.809226
1,19077.0,0.190774,99998.0,1.000000


In [11]:
freq(merged["sbfehitindex_flag"])

,Count,Pct,Cuml Count,Cuml Pct
sbfehitindex_flag,,,,
0,99988.0,0.9999,99988.0,0.9999
1,10.0,0.0001,99998.0,1.0000


In [12]:
freq(merged["model2score_flag"])

,Count,Pct,Cuml Count,Cuml Pct
model2score_flag,,,,
0,81072.0,0.810736,81072.0,0.810736
1,18926.0,0.189264,99998.0,1.000000


In [13]:
exclude = ['accountnumber', 'historydateyyyymm', 'bus_company_name', 'powid', 'proxid', 'seleid', 'orgid', 'ultid', 'lnlexidsele','model1name', 'model1score',
           'model1rc1', 'model1rc2', 'model1rc3', 'model1rc4', 'model1rc5', 'model1rc6', 'model2name', 'model2score', 'model2rc1', 'model2rc2', 'model2rc3', 'model2rc4',
           'model2rc5', 'model2rc6', 'model3name', 'model3score', 'model3rc1', 'model3rc2', 'model3rc3', 'model3rc4', 'model3rc5', 'model3rc6', 'model4name', 'model4score',
           'model4rc1', 'model4rc2', 'model4rc3', 'model4rc4', 'model4rc5', 'model4rc6', 'model5name', 'model5score', 'model5rc1', 'model5rc2', 'model5rc3', 'model5rc4',
           'model5rc5', 'model5rc6', 'model6name', 'model6score', 'model6rc1', 'model6rc2', 'model6rc3', 'model6rc4', 'model6rc5', 'model6rc6', 'model7name', 'model7score',
           'model7rc1', 'model7rc2', 'model7rc3', 'model7rc4', 'model7rc5', 'model7rc6', 'model8name', 'model8score', 'model8rc1', 'model8rc2', 'model8rc3', 'model8rc4',
           'model8rc5', 'model8rc6', 'model9name', 'model9score', 'model9rc1', 'model9rc2', 'model9rc3', 'model9rc4', 'model9rc5', 'model9rc6', 'model10name', 'model10score',
           'model10rc1', 'model10rc2', 'model10rc3', 'model10rc4', 'model10rc5', 'model10rc6', 'errorcode', 'time_ms']
sbfe_attr = [x for x in baseline.columns if "sbfe" in x]
rv_attr = [x for x in baseline.columns if x not in sbfe_attr and x not in exclude]
model = ["model1score", "model2score"]

exception_dict = dict()
for x in baseline.columns:
    if x in sbfe_attr:
        exception_dict[x] = [-99, -98, -97]
    if x in rv_attr:
        exception_dict[x] = [-1]
    if x in model:
        exception_dict[x] = [0, 100, 200, 222]

In [14]:

from dsgtools.reporting import ImpactAnalysisReport
fmt_score = make_format(cuts = [-np.inf, 500, 550, 600, 625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 900, np.inf], exceptions = [0, 100, 200, 222])
cr = ImpactAnalysisReport(baseline, test_1, keys="accountnumber",
           score_formats = {"model1score" : fmt_score, 
                         'model2score': fmt_score, },
           score_columns = ["model1score", "model2score", "sbfehitindex"], exceptions = exception_dict,
           dataframe_names = ("baseline", "test1"))
cr.to_xlsx("./_temp/air937_201506_test1_data-comparison.xlsx", overwrite= True)

<Figure size 640x480 with 0 Axes>

In [15]:
fmt_score = make_format(cuts = [-np.inf, 500, 550, 600, 625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 900, np.inf], exceptions = [0, 100, 200, 222])
cr = ImpactAnalysisReport(baseline, test_2, keys="accountnumber",
           score_columns = ["model1score", "model2score", "sbfehitindex"], 
           score_formats = {"model1score" : fmt_score, 
                         'model2score': fmt_score, },
           exceptions=exception_dict, 
           dataframe_names = ("baseline", "test2"))
cr.to_xlsx("./_temp/air937_201506_test2_data-comparison.xlsx", overwrite= True)

<Figure size 640x480 with 0 Axes>

In [16]:

# from dsgtools.reporting import ImpactAnalysisReport
# fmt_score = make_format(cuts = [-np.inf, 500, 550, 600, 625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 900, np.inf], exceptions = [0, 100, 200, 222])
# cr = ImpactAnalysisReport(test_1, test_2, keys="accountnumber",
#            score_columns = ["model1score", "model2score", "sbfehitindex"], 
#            score_formats = {"model1score" : fmt_score, 
#                          'model2score': fmt_score, },
#            exceptions=exception_dict, 
#            dataframe_names = ("test_1", "test2"))
# cr.to_xlsx("./_temp/temp_air937_201506_test1_test2_data-comparison.xlsx", overwrite= True)